In [1]:
import yfinance as yf
import re
import pandas as pd
from pandas import DataFrame, Series

In [5]:
%pylab inline
from datetime import datetime
#from yahoofinancials import YahooFinancials

Populating the interactive namespace from numpy and matplotlib


In [28]:
SP = yf.Ticker('ES=F')

In [29]:
df = SP.history(period = '7d', interval = '1m')

In [30]:
df.shape

(8075, 7)

In [31]:
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2021-08-25 00:00:00-04:00,4478.25,4478.25,4478.25,4478.25,0,0,0
2021-08-25 00:01:00-04:00,4478.25,4478.25,4478.25,4478.25,43,0,0
2021-08-25 00:02:00-04:00,4478.25,4478.25,4478.25,4478.25,78,0,0
2021-08-25 00:03:00-04:00,4478.25,4478.25,4478.25,4478.25,31,0,0
2021-08-25 00:04:00-04:00,4478.25,4478.25,4478.25,4478.25,105,0,0
...,...,...,...,...,...,...,...
2021-09-01 23:15:00-04:00,4517.75,4517.75,4517.50,4517.75,45,0,0
2021-09-01 23:16:00-04:00,4518.00,4518.00,4517.25,4517.25,23,0,0
2021-09-01 23:17:00-04:00,4517.50,4517.50,4517.00,4517.50,185,0,0


In [32]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.keras.layers import LeakyReLU

In [48]:
RNN_df = df[['Close', 'Volume']]
RNN_df

,Close,Volume
Datetime,,
2021-08-25 00:00:00-04:00,4478.25,0
2021-08-25 00:01:00-04:00,4478.25,43
2021-08-25 00:02:00-04:00,4478.25,78
2021-08-25 00:03:00-04:00,4478.25,31
2021-08-25 00:04:00-04:00,4478.25,105
...,...,...
2021-09-01 23:15:00-04:00,4517.75,45
2021-09-01 23:16:00-04:00,4517.25,23
2021-09-01 23:17:00-04:00,4517.50,185


In [51]:
#notes for prediction
seq_len = 60
future_period_predict = 3
ratio_to_predict= 'ES' 

#classification target
def classify(current, pct_change):
    tot_change=0
    for x in RNN_df['pct_change']:
        tot_change = tot_change + x
        if tot_change >= 0.01:
            return 1
        elif tot_change<= 0.005:
            return 0 
        elif tot_change<0:
            tot_change=0
            continue
        else:
            continue
#shift timeseries down 3 spots for 3 hour prediction
pct_change = ((RNN_df['Close'].shift(-future_period_predict))-RNN_df['Close'])/RNN_df['Close']

#create a 3 hour future column
RNN_df["pct_change"] = pct_change
RNN_df

<ipython-input-51-8a8e666dfe96>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RNN_df["pct_change"] = pct_change


,Close,Volume,pct_change,target
Datetime,,,,
2021-08-25 00:00:00-04:00,4478.25,0,0.000000,1
2021-08-25 00:01:00-04:00,4478.25,43,0.000000,1
2021-08-25 00:02:00-04:00,4478.25,78,0.000000,1
2021-08-25 00:03:00-04:00,4478.25,31,0.000000,1
2021-08-25 00:04:00-04:00,4478.25,105,0.000000,1
...,...,...,...,...
2021-09-01 23:12:00-04:00,4517.75,32,0.000000,1
2021-09-01 23:13:00-04:00,4517.50,140,-0.000055,1
2021-09-01 23:14:00-04:00,4517.50,52,NaN,1


In [52]:
RNN_df=RNN_df.dropna()

#send close and 3d future to classify function
def grab_arguments(s):
    return classify(s[0],s[1])

#create a classification
RNN_df['target'] = RNN_df[['Close','pct_change']].T.apply(grab_arguments)


<ipython-input-52-24fd728daf53>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RNN_df['target'] = RNN_df[['Close','pct_change']].T.apply(grab_arguments)


In [57]:
RNN_df['target']==1



Datetime
2021-08-25 00:00:00-04:00    False
2021-08-25 00:01:00-04:00    False
2021-08-25 00:02:00-04:00    False
2021-08-25 00:03:00-04:00    False
2021-08-25 00:04:00-04:00    False
                             ...  
2021-09-01 23:09:00-04:00    False
2021-09-01 23:10:00-04:00    False
2021-09-01 23:11:00-04:00    False
2021-09-01 23:12:00-04:00    False
2021-09-01 23:13:00-04:00    False
Name: target, Length: 8069, dtype: bool

In [64]:

mask= (RNN_df['target'] == 1)
RNN_df[mask]

,Close,Volume,3 hour future,target,pct_change
Datetime,,,,,


In [15]:
################################
##split into train and test set
#############################

#make sure index is sorted. probs already was
times = sorted(RNN_df.index)

# last 5% of time as test set
last_10pct = times[-int(0.1*len(times))]
print(last_10pct)


2021-09-01 11:21:00-04:00


In [16]:
# test set is last 10% and beyond
test_set = RNN_df[RNN_df.index >= last_10pct]
test_set
#train set is anything under the last 10%
train_set =RNN_df[(RNN_df.index < last_10pct)]
train_set

,Close,Volume,3 hour future,target
Datetime,,,,
2021-08-24 09:30:00-04:00,90.860001,9689,90.080002,0
2021-08-24 09:31:00-04:00,90.759903,558,90.190002,0
2021-08-24 09:33:00-04:00,90.730003,274,90.119003,0
2021-08-24 09:34:00-04:00,90.559998,1058,90.220001,0
2021-08-24 09:35:00-04:00,90.449997,174,90.139999,0
...,...,...,...,...
2021-09-01 11:16:00-04:00,91.489998,2377,91.364998,0
2021-09-01 11:17:00-04:00,91.485001,7714,91.410004,0
2021-09-01 11:18:00-04:00,91.440002,3129,91.470001,1


In [17]:
#Drop the right answer, 3 hour future
train_set = train_set.T.drop(['3 hour future']).T
test_set = test_set.T.drop(['3 hour future']).T
len(train_set)
len(test_set)

265

In [18]:
#test_set

In [19]:
## check for infinte values in your data

#train_set['Volume'].preprocessing.scale(train_set['Volume'])
import numpy as np
#check for inf as well as -inf
#mask1 = isinf(-train_set['Volume'])
#train_set[mask1]


#mask = isinf(test_set['Volume'])
#test_set[mask]

# mask = train_set['Volume'] == float(inf)
# train_set[mask]= 0

train_set.loc[(train_set['Volume'] == float("inf")),'Volume']=0
train_set[train_set["Volume"] == 0]

# Replacing infinite with nan
#train_set.replace([np.inf, -np.inf], np.nan, inplace=True)
  
# Dropping all the rows with nan values
#train_set.dropna(inplace=True)
  
# Printing df


# Replacing infinite with nan
#test_set.replace([np.inf, -np.inf], np.nan, inplace=True)
  
# Dropping all the rows with nan values
#test_set.dropna(inplace=True)
  
# Printing df
#len(test_set)
#len(train_set)
#find the infinite values to get rid of
# 2021-05-05 13:30:00-04:00
#train_set.T["2021-05-05 13:30:00-04:00"]
#drop that index or indices
#RNN_df=RNN_df[~RNN_df.isin([np.nan, np.inf, -np.inf]).any(1)]
#len(RNN_df)

,Close,Volume,target
Datetime,,,
2021-08-31 09:32:00-04:00,90.669998,0.0,1.0


In [20]:
mask = (test_set['Volume'] == float("inf"))
test_set[mask]

,Close,Volume,target
Datetime,,,


In [21]:
#train_set = train_set.drop('2019-12-02 09:30:00-05:00')
#train_set = train_set.drop('2019-12-26 09:30:00-05:00')
#train_set = train_set.drop('2020-05-14 12:30:00-04:00')
#train_set = train_set.drop('2020-12-28 09:30:00-05:00')
#train_set = train_set.drop('2021-05-05 14:30:00-04:00')
#train_set = train_set.drop('2020-11-30 09:30:00-05:00')

In [22]:
###################
#Preprocessing - scale and normalize
#####################

#import preprocessing
from sklearn import preprocessing

# import deque continual appending to list and maintains list len
from collections import deque

import numpy as np
import random

def preprocess_df(df):
    df.dropna(inplace=True)
    
    for col in df:
        
        if col != "target":
            #normalize with % change 
            df[col] = df[col].pct_change()
            df.dropna(inplace=True)
            #scale the data
            df.loc[(df['Volume'] == float("inf")),'Volume']=0
            #print(df[df[col] == float("inf")])
            df[col]= preprocessing.scale(df[col].values)
        
    
    df.dropna(inplace = True)
    
    sequential_data = []
    #maintain the seq_len
    prev_days = deque(maxlen=seq_len)
    #turns df into a list of lists
    for i in df.values:
        # take each value except for target col in list
        prev_days.append([n for n in i[:-1]])
        #print(len(prev_days))
        if len(prev_days) == seq_len:
            sequential_data.append([np.array(prev_days), i[-1]])
            #print(sequential_data)
    
    random.shuffle(sequential_data)  
    
    #####################
    ### Balancet the Data 
    ######################

    buys = []
    sells = []
    for seq, target in sequential_data:
        if target == 0:
            sells.append([seq,target])
        elif target ==1:
            buys.append([seq,target])
     
    random.shuffle(buys)
    random.shuffle(sells)
    
    lower = min(len(buys), len(sells)) 
    buys = buys[:lower]
    sells = sells[:lower]
    #print(sells)
    sequential_data = buys+sells
    
    random.shuffle(sequential_data)
    
    x = []
    y = []
    
    for seq, target in sequential_data:
        x.append(seq)
        y.append(target)
        
    return np.array(x), y 

#preprocess_df(train_set)
train_x, train_y = preprocess_df(train_set)
test_x, test_y = preprocess_df(test_set)

In [23]:
print(f"train data:{len(train_x)} validation: {len(test_x)}")

train data:2224 validation: 198


In [24]:
print(f"Dont buys:{train_y.count(0)}, buys:{train_y.count(1)}")

Dont buys:1112, buys:1112


In [25]:
print(f"VALIDATION Dont buys: {test_y.count(0)}, buys: {test_y.count(1)}")

VALIDATION Dont buys: 99, buys: 99


In [26]:

epochs = 15
batch_size = 64
name = f"{seq_len}-SEQ-{future_period_predict}-PRED-{int(time.time())}"



In [27]:
# check to make sure training and test set are numpy arrays
test_y = np.asarray(test_y)
train_y = np.asarray(train_y)

In [28]:
model = Sequential()
model.add(LSTM(128, input_shape= (train_x.shape[1:]), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(LSTM(128, input_shape= (train_x.shape[1:]), return_sequences=True))
model.add(Dropout(0.1))
model.add(BatchNormalization())

model.add(LSTM(128, input_shape= (train_x.shape[1:])))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(tf.keras.layers.Flatten())
#dense layer activation rectified linear


model.add(Dense(32, activation ='relu'))
model.add(Dropout(0.2))

#output layer
model.add(Dense(2, activation = 'softmax'))

opt = tf.keras.optimizers.SGD(lr= 0.0005, decay=1e-3)
model.compile(loss='sparse_categorical_crossentropy',
             optimizer = opt,
             metrics= ['accuracy'])

tensorboard = TensorBoard(log_dir = f'log/{name}')


checkpoint = ModelCheckpoint(filepath= r'C:\Users\chand\My Python\models',verbose=1, save_best_only=True)

history = model.fit(
            train_x, train_y,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(test_x, test_y),
            callbacks=[tensorboard, checkpoint])
#model.save('')

Epoch 1/15
 1/35 [..............................] - ETA: 0s - loss: 1.3884 - accuracy: 0.4688WARNING:tensorflow:From C:\Users\chand\Anaconda3\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
35/35 [==============================] - ETA: 0s - loss: 1.1826 - accuracy: 0.4969
Epoch 00001: val_loss improved from inf to 0.69280, saving model to C:\Users\chand\My Python\models
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: C:\Users\chand\My Python\models\assets
35/35 [==============================] - 87s 2s/step - loss: 1.1826 - accuracy: 0.4969 - val_loss: 0.6928 - val_accuracy: 0.5505
Epoch

KeyboardInterrupt: 